## Capstone Project - Startup Investments

### Part 1 - Data Cleaning/ Preprocessing

### Problem Statement

The main goal of this project is to classify whether a startup’s current status is in operating status, acquired status or closed status using various features variables in investment venture capital data set.

**Please note :** There are various other factors such as the quality of the management, quality of the product, price points, revenue, expenses, tangible and intangible assets etc. that determines the status of a startup but for this project we will not be considering them since we don’t have the required data and also the main goal of this project is to show the entire Machine Learning pipeline for a non-traditional data set.

In [1]:
## Data handling Libraries ###

import pandas as pd
import numpy as np

## Plotting Libraries ###
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

## Date Time ###
import datetime
import time
import pytz

### Warnings ###
import warnings
warnings.filterwarnings('ignore')

### Progress Bar ###
from tqdm import tqdm

### Model Building, Model Evaluvation, Model Preprocessing ###
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict,RepeatedStratifiedKFold,StratifiedKFold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import VarianceThreshold,RFECV

### Models Imbalance #

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# ML MODELS #

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier

# Scoring Dependancies #

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score 
# from sklearn.metrics import average_precision_score,make_scorer
from sklearn.model_selection import cross_val_score, cross_validate, KFold
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix

# Models Saving #

import pickle

# Other #
from collections import Counter
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('../dataset/investments_VC.csv')

### Data Dictionary

Please see below for feature variables that will be used for predicting the target class of a startup and after understanding the features I think you will be convinced that the below mentioned features are sufficient to predict the current status of a startup.

1. Permalink
    - This attribute gives us the link of the CrunchBase website where the information about this particular startup could be found.
    
2. Name
    - This attribute gives us the name of the startup.

3. homepage_url
    - This attribute gives us the homepage URL of the startup.
    
4. category_list
    - This attribute helps us to know in which category the startup falls. Ex. Mobile, Artificial Intelligence, Recommendations, Medical etc. Please note that a particular startup can fall under multiple categories as well.

5. Market
    - This attribute helps us to know which market the startup caters to. Ex. Hospitality, Real Estate, Mobile etc.

6. funding_total_usd
    - This attribute helps us to know the total funding received by the startup.

7. Status
    - This is the target variable that we are trying to predict. It tells whether the startup is in operating status, acquired status or closed status.

**Location Features**

The below four variables give us information regarding the location of the startup.

8. country_code
9. state_code
10. region
11. city
12. funding_rounds
    - This variable tells us the total number of funding rounds a startup has raised.

**Date Features**

The below four variables gives us the information about the date in which the startup was founded.

13. founded_at
14. founded_month
15. founded_quarter
16. founded_year
17. first_funding_at
    - This variable gives us information about the date of first funding raised.
18. last_funding_at
    - This variable gives us information about the date of last funding raised.
19. Seed
    - This feature gives us information about total seed funding received (in USD) by the startup.
20. Venture
    - This feature gives us information about total venture funding received (in USD) by the startup. 
21. equity_crowdfunding
    - This feature gives us information about funds received by diluting the equity of startup. 
22. Undisclosed
    - This feature gives us information about total funds received by the startup from other undisclosed funding sources.
23. convertible_note
    - This feature gives us information about funding raised through convertible note option by the startup. 
24. debt_financing
    - This feature gives us information about funding raised as debt by the startup. 
25. Angel
    - This feature gives us information about funding received from angel investors. 
26. Grant
    - This feature gives us information about funding received as a grant. 
27. private_equity
    - This feature gives us information about funding received from private equity investors. 
28. post_ipo_equity
    - This feature gives us information about funding through equity dilution after IPO. 
29. post_ipo_debt
    - This feature gives us information about funding raised as debt after IPO. 
30. secondary_market
    - This feature gives us information about funding raised from secondary markets. 
31. product_crowdfunding
    - This feature gives us information about funding via product crowdfunding route. 

**Funding Rounds**

The below features help us to know the funding raised by a startup in each funding rounds. 

32. round_A
33. round_B
34. round_C
35. round_D
36. round_E
37. round_F
38. round_G
39. round_H

In [3]:
pd.set_option('display.max_rows', 40)
pd.set_option('display.max_columns', 1000)

In [5]:
def funcPreprocessing(startup_df):
    
    '''
    This function
    1)Removes leading and trailing white spaces in the column names.
    2)Removes leading and trailing white spaces in the values present in object datatype columns.
    
    '''

    dictCol = {}
    
    print('Column preprocessing...')
    for col in df.columns:
        dictCol[col] = col.strip()
    df.rename(columns=dictCol, inplace=True)
    
    print('Object datatype preprocessing...\n')
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    
    print('Sucessfully preprocessed the dataframe!')  

In [6]:
funcPreprocessing(df)

Column preprocessing...
Object datatype preprocessing...

Sucessfully preprocessed the dataframe!


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54294 entries, 0 to 54293
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permalink             49438 non-null  object 
 1   name                  49437 non-null  object 
 2   homepage_url          45989 non-null  object 
 3   category_list         45477 non-null  object 
 4   market                45470 non-null  object 
 5   funding_total_usd     49438 non-null  object 
 6   status                48124 non-null  object 
 7   country_code          44165 non-null  object 
 8   state_code            30161 non-null  object 
 9   region                44165 non-null  object 
 10  city                  43322 non-null  object 
 11  funding_rounds        49438 non-null  float64
 12  founded_at            38554 non-null  object 
 13  founded_month         38482 non-null  object 
 14  founded_quarter       38482 non-null  object 
 15  founded_year       

In [8]:
df.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', 'market',
       'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'seed', 'venture', 'equity_crowdfunding',
       'undisclosed', 'convertible_note', 'debt_financing', 'angel', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H'],
      dtype='object')

In [9]:
df.isnull().sum()

permalink                4856
name                     4857
homepage_url             8305
category_list            8817
market                   8824
funding_total_usd        4856
status                   6170
country_code            10129
state_code              24133
region                  10129
city                    10972
funding_rounds           4856
founded_at              15740
founded_month           15812
founded_quarter         15812
founded_year            15812
first_funding_at         4856
last_funding_at          4856
seed                     4856
venture                  4856
equity_crowdfunding      4856
undisclosed              4856
convertible_note         4856
debt_financing           4856
angel                    4856
grant                    4856
private_equity           4856
post_ipo_equity          4856
post_ipo_debt            4856
secondary_market         4856
product_crowdfunding     4856
round_A                  4856
round_B                  4856
round_C   

#### Checking and deleting rows where all the values are Nan(Missing)

The below block of code checks if all the values in a row is NaN and if yes, drops them from our analysis. There were 4856 rows that had only NaN as its values and hence we have removed these records from our analysis.

In [10]:
na_values = df[df.isna().all(axis = 1)].shape[0]
print(f'There are {na_values} rows that have only NaN as values')

There are 4856 rows that have only NaN as values


In [11]:
df[df.isna().all(axis = 1)]

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
49438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df = df.dropna(how = 'all')
df = df.reset_index(drop = True)

In [13]:
#Drop 4856 rows which all contain NaN(Missing values) out of 54294 rows
df.shape

(49438, 39)

#### Checking for duplicate values

In [14]:
df[df.duplicated()]

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H


In [15]:
dup_values = df[df.duplicated()].shape[0]
if dup_values == 0:
    print(f'There are {dup_values} duplicat values in the dataframe')
else:
    print(f'There are {dup_values} duplicat values are dropped in the dataframe')

There are 0 duplicat values in the dataframe


#### Dropping rows where target variables 'Status' is NaN

In this project I have dropped the rows for which we are missing the target variable. There were 1314 rows for which target variable had NaN as its value.

In [16]:
df.loc[df['status'].isna() == True,]

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
5,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,"70,00,000",NaN,USA,FL,Ft. Lauderdale,Oakland Park,1.0,2011-10-10,2011-10,2011-Q4,2011.0,2013-05-31,2013-05-31,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0
67,/organization/1c-company,1C Company,http://1c.ru/eng,|Video Games|Games|Software|,Software,"20,00,00,000",NaN,RUS,NaN,Moscow,Moscow,1.0,1991-01-01,1991-01,1991-Q1,1991.0,2011-10-03,2011-10-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,/organization/1st-merchant-funding,1st Merchant Funding,http://www.1stmerchantfunding.com/,|Financial Services|,Financial Services,"1,00,00,000",NaN,USA,FL,Miami,Miami,1.0,2007-01-01,2007-01,2007-Q1,2007.0,2014-05-07,2014-05-07,0.0,0.0,0.0,0.0,0.0,10000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,/organization/24h00,24h00,http://www.boosket.com/,NaN,NaN,-,NaN,FRA,NaN,Paris,Paris,1.0,2006-01-01,2006-01,2006-Q1,2006.0,2006-08-01,2006-08-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
238,/organization/3nder,3nder,http://www.3nderapp.com,|iOS|Online Dating|Mobile|Location Based Servi...,Social Media,-,NaN,NaN,NaN,NaN,NaN,1.0,2014-02-18,2014-02,2014-Q1,2014.0,2014-05-09,2014-05-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49175,/organization/zingaya,Zingaya,http://www.zingaya.com,|Customer Service|E-Commerce|VoIP|Telecommunic...,Messaging,"11,50,000",NaN,USA,CA,SF Bay Area,Palo Alto,1.0,2010-06-01,2010-06,2010-Q2,2010.0,2011-10-26,2011-10-26,0.0,1150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49248,/organization/tyrosine-pharmaceuticals,Zocere,http://zocere.com/,|Biotechnology|,Biotechnology,"1,00,000",NaN,NaN,NaN,NaN,NaN,1.0,2012-01-01,2012-01,2012-Q1,2012.0,2013-09-25,2013-09-25,0.0,100000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49288,/organization/zoojoo-be,zoojoo.BE,http://www.zoojoo.be,|Software|,Software,-,NaN,IND,NaN,Bangalore,Bangalore,1.0,2012-01-01,2012-01,2012-Q1,2012.0,2014-05-15,2014-05-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49289,/organization/zookal,Zookal,http://www.zookal.com,|Education|Textbooks|E-Commerce|,E-Commerce,"20,60,000",NaN,AUS,NaN,Sydney,Sydney,3.0,2011-03-01,2011-03,2011-Q1,2011.0,2011-12-01,2013-09-30,260000.0,1800000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1800000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
target_values = df.loc[df['status'].isna() == True,].shape[0]
print(f'There are {target_values} rows for which target variable has NaN as values')
df = df.loc[df['status'].isna() == False,]

There are 1314 rows for which target variable has NaN as values


In [18]:
df.shape

(48124, 39)

### Constants

In [19]:
LISTCOLUMNNAME= []
LISTFUNDINGCOL = ['seed', 'venture',
       'equity_crowdfunding', 'undisclosed', 'convertible_note',
       'debt_financing', 'grant', 'private_equity', 'post_ipo_equity',
       'post_ipo_debt', 'secondary_market', 'product_crowdfunding', 'round_A',
       'round_B', 'round_C', 'round_D', 'round_E', 'round_F', 'round_G',
       'round_H']

#### Removing the rows for which we don’t have total funding and the breakup of the funding is null as well

In [20]:
df.loc[(df['funding_total_usd'] == '-'),['funding_total_usd']]

,funding_total_usd
8,-
11,-
12,-
14,-
19,-
...,...
49411,-
49418,-
49421,-
49422,-


In [21]:
print('The list of variables associated with funding are \n', LISTFUNDINGCOL)
df['f_SumCol'] = df[LISTFUNDINGCOL].sum(axis = 1)

rowsTotalSumCheck = df.loc[(df['funding_total_usd'] == '-'),['funding_total_usd']].shape[0]
print(f'There are {rowsTotalSumCheck} rows whose total funding is none ("-")')

rowsTotalSumCheck = df.loc[(df['funding_total_usd'] == '-') & (df['f_SumCol'] > 0),].shape[0]
print(f'There are {rowsTotalSumCheck} rows whose total funding is none but sum of the total funding is more than zero')

### Selecting only the rows for which the variable funding_total_usd is not equal to '-' ###
df = df.loc[~(df['funding_total_usd'] == '-'),]

The list of variables associated with funding are 
 ['seed', 'venture', 'equity_crowdfunding', 'undisclosed', 'convertible_note', 'debt_financing', 'grant', 'private_equity', 'post_ipo_equity', 'post_ipo_debt', 'secondary_market', 'product_crowdfunding', 'round_A', 'round_B', 'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H']
There are 8322 rows whose total funding is none ("-")
There are 0 rows whose total funding is none but sum of the total funding is more than zero


In [22]:
df.shape

(39802, 40)

#### Rows with high NaN percentage

It is always best to check the percentage of missing values in a particular row and in a particular feature variable (column) and drop them if it is higher than a certain threshold value. Usually I drop a row or a column if the percentage of missing values is greater than 80%.

Even though some people can argue that you can impute the missing value but I personally feel that if 80% of the data is missing, then it will be erroneous to impute the missing value since majority of the information is not available.

In [23]:
lstHighNanRow = []
for i in tqdm(range(len(df.index)), desc='Rows processed'):
    temp = round((df.iloc[i].isnull().sum()/df.shape[1])*100,2)
    if temp >= 80:
      lstHighNanRow.append(i)
print(f'\nNumber of rows deleted is {len(lstHighNanRow)}')
df = df.drop(df.index[lstHighNanRow])

Rows processed: 100%|██████████| 39802/39802 [00:04<00:00, 9536.76it/s]


Number of rows deleted is 0


In our analysis there were no rows whose NaN percentage was higher than 80%.

#### Removing Columns that don't contribute to model building

During our analysis we need to check the uniqueness percent of a column. If a particular feature falls in the above mentioned first two categories then it is better to drop the feature altogether since in the first instance all the records carry the same value — the entropy is zero and in the second case all the values are different — the entropy will be on a higher side.

Below custom function helps to drop

- “permalink” and “name” feature variables since all the values in these columns are unique.

- “state_code” feature variable since this feature has the maximum number of null values and also the remaining non-null values belong to only two categories — US and Canada.

- “city” feature variable since this feature has very high unique category count (3536 unique cities) and hence high uniqueness percent (unique count/total count). Please note that this feature is of object (string) datatype and if we need to use them in our analysis, we need to one-hot encode the feature which will induce curse of dimensionality.

In [24]:
def funcColumnsToDrop(data, drop_list = []):
    
    '''
    This function drops a list of column(s) in the dataframe and returns the new unique columns in the dataframe
    input: Dataframe, list of coulumns to drop
    output: list of new columns (Sucess) / -1 (Failure)
    '''
    
    if len(drop_list) > 0 and set(drop_list).issubset(data.columns):
        data.drop(drop_list, axis=1, inplace=True) 
        return data.columns
    else:
        print('No columns to drop or one of the columns present in the list is not available in the dataframe')
        return -1

In [25]:
### Removing permalink and name since this would not contribute to the model ###

drop_list = ['permalink', 'name']
column_list = funcColumnsToDrop(df, drop_list)

In [26]:
### Dropping region based columns ###

# Replacing Nan with 'Unknown' where all the region based columns have NaN values 
lstcolumns = ['country_code', 'state_code','region', 'city']
df.loc[(df[lstcolumns].isna().all(axis=1) == True), lstcolumns] = 'Unknown'

txt = 'Total Null values is '
print('country_code')
print(txt, df[ 'country_code'].isnull().sum())
print('\n')
print('state_code')
print(txt, df[ 'state_code'].isnull().sum())
print('\n')
print('region')
print(txt, df[ 'region'].isnull().sum())
print('\n')
print('city')
print(txt, df[ 'city'].isnull().sum())
print('\n')

print(df.loc[(df['state_code'].isnull() == False), lstcolumns]['country_code'].value_counts())

# Since the state_code has the maximum number of null values and remaing non-null values belong to US and Canada,
# we will be removing the state_code variable

lstDropColumns = ['state_code']
LISTCOLUMNNAME = funcColumnsToDrop(df, lstDropColumns)

country_code
Total Null values is  0


state_code
Total Null values is  11039


region
Total Null values is  0


city
Total Null values is  635


USA        23995
Unknown     3663
CAN         1105
Name: country_code, dtype: int64


In [27]:
### Dropping City column ###

city_count = len(df['city'].unique())
ratio_percent = round(city_count/len(df['city'])*100,2)
print(f'There are {city_count} unique cities and hence dropping the column since uniquness ratio is low ({ratio_percent})')

lstDropColumns = ['city']
LISTCOLUMNNAME = funcColumnsToDrop(df, lstDropColumns)

There are 3536 unique cities and hence dropping the column since uniquness ratio is low (8.88)


#### Removing redundant features

Need to remove redundant features and highly correlated features from our analysis because

- Few models require the features to be truly independent.
- Help us in saving resources by reducing the data set size without losing any crucial information.

In [28]:
### Removing funding_total_usd since this is just an addition of all the funding columns###

lstDropColumns = ['funding_total_usd'] #, 'f_SumCol']
LISTCOLUMNNAME = funcColumnsToDrop(df, lstDropColumns)

### Combining Seed investment and Angel Investment into one since both are the same.

df['seed'] = df['seed'] + df['angel']
lstDropColumns = ['angel']
LISTCOLUMNNAME = funcColumnsToDrop(df, lstDropColumns)

In [29]:
df.columns

Index(['homepage_url', 'category_list', 'market', 'status', 'country_code',
       'region', 'funding_rounds', 'founded_at', 'founded_month',
       'founded_quarter', 'founded_year', 'first_funding_at',
       'last_funding_at', 'seed', 'venture', 'equity_crowdfunding',
       'undisclosed', 'convertible_note', 'debt_financing', 'grant',
       'private_equity', 'post_ipo_equity', 'post_ipo_debt',
       'secondary_market', 'product_crowdfunding', 'round_A', 'round_B',
       'round_C', 'round_D', 'round_E', 'round_F', 'round_G', 'round_H',
       'f_SumCol'],
      dtype='object')

In [30]:
#Exporting Data Cleaning to .CSV
df.to_csv('../dataset/data_cleaning.csv', index = False)